# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
load_csv = "output_data/cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bethel,41.37,-73.41,44.60,93,90,11.41,US,1970-01-01 00:00:01.604024598
1,1,Tasiilaq,65.61,-37.64,35.60,64,100,24.16,GL,1970-01-01 00:00:01.604024599
2,2,Punta Arenas,-53.15,-70.92,44.60,75,75,5.82,CL,1970-01-01 00:00:01.604024391
3,3,La Santisima Trinidad,-14.83,-64.90,72.99,80,100,6.87,BO,1970-01-01 00:00:01.604024527
4,4,Vaini,-21.20,-175.20,77.00,65,20,9.17,TO,1970-01-01 00:00:01.604024386


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Lat and Lng as locations and Humidity as the weight
locations = city_df[["Lat", "Lng"]]
humidity = city_df['Humidity'].astype(int)


# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down the cities to fit weather conditions.

narrowed_city_df = city_df.copy()
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= 70]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= 80]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= 10]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]

#Drop any rows will null values
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()
narrowed_city_df

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,14,14,Karwar,14.80,74.13,76.14,71,0,6.69,IN,1970-01-01 00:00:01.604024619
1,35,35,Airai,-8.93,125.41,77.92,48,0,2.62,TL,1970-01-01 00:00:01.604024649
2,50,50,Doha,25.29,51.53,77.00,69,0,2.24,QA,1970-01-01 00:00:01.604024596
3,109,109,Tessalit,20.20,1.01,73.26,20,0,5.28,ML,1970-01-01 00:00:01.604024738
4,153,153,Jega,12.22,4.38,78.67,25,0,7.07,NG,1970-01-01 00:00:01.604024794
5,169,169,Wanlaweyn,2.62,44.89,75.61,88,0,3.49,SO,1970-01-01 00:00:01.604024817
6,171,171,Mau,25.95,83.55,71.94,46,0,4.00,IN,1970-01-01 00:00:01.604024732
7,245,245,Kassala,15.45,36.40,76.50,54,0,7.20,SD,1970-01-01 00:00:01.604024915
8,266,266,Tāndā,26.55,82.65,71.76,57,0,5.06,IN,1970-01-01 00:00:01.604024940
9,274,274,Doka,13.52,35.77,75.13,44,0,5.17,SD,1970-01-01 00:00:01.604024949


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Store into variable named hotel_df
hotel_df = narrowed_city_df
hotel_df["Hotel Name"] = np.nan
hotel_df = hotel_df.drop(columns='index')
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,14,Karwar,14.80,74.13,76.14,71,0,6.69,IN,1970-01-01 00:00:01.604024619,NaN
1,35,Airai,-8.93,125.41,77.92,48,0,2.62,TL,1970-01-01 00:00:01.604024649,NaN
2,50,Doha,25.29,51.53,77.00,69,0,2.24,QA,1970-01-01 00:00:01.604024596,NaN
3,109,Tessalit,20.20,1.01,73.26,20,0,5.28,ML,1970-01-01 00:00:01.604024738,NaN
4,153,Jega,12.22,4.38,78.67,25,0,7.07,NG,1970-01-01 00:00:01.604024794,NaN
5,169,Wanlaweyn,2.62,44.89,75.61,88,0,3.49,SO,1970-01-01 00:00:01.604024817,NaN
6,171,Mau,25.95,83.55,71.94,46,0,4.00,IN,1970-01-01 00:00:01.604024732,NaN
7,245,Kassala,15.45,36.40,76.50,54,0,7.20,SD,1970-01-01 00:00:01.604024915,NaN
8,266,Tāndā,26.55,82.65,71.76,57,0,5.06,IN,1970-01-01 00:00:01.604024940,NaN
9,274,Doka,13.52,35.77,75.13,44,0,5.17,SD,1970-01-01 00:00:01.604024949,NaN


In [6]:
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "hotel",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_data = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = "Nearest Hotel Not Found"

hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,14,Karwar,14.80,74.13,76.14,71,0,6.69,IN,1970-01-01 00:00:01.604024619,Nearest Hotel Not Found
1,35,Airai,-8.93,125.41,77.92,48,0,2.62,TL,1970-01-01 00:00:01.604024649,Nearest Hotel Not Found
2,50,Doha,25.29,51.53,77.00,69,0,2.24,QA,1970-01-01 00:00:01.604024596,Nearest Hotel Not Found
3,109,Tessalit,20.20,1.01,73.26,20,0,5.28,ML,1970-01-01 00:00:01.604024738,Nearest Hotel Not Found
4,153,Jega,12.22,4.38,78.67,25,0,7.07,NG,1970-01-01 00:00:01.604024794,Nearest Hotel Not Found
5,169,Wanlaweyn,2.62,44.89,75.61,88,0,3.49,SO,1970-01-01 00:00:01.604024817,Nearest Hotel Not Found
6,171,Mau,25.95,83.55,71.94,46,0,4.00,IN,1970-01-01 00:00:01.604024732,Nearest Hotel Not Found
7,245,Kassala,15.45,36.40,76.50,54,0,7.20,SD,1970-01-01 00:00:01.604024915,Nearest Hotel Not Found
8,266,Tāndā,26.55,82.65,71.76,57,0,5.06,IN,1970-01-01 00:00:01.604024940,Nearest Hotel Not Found
9,274,Doka,13.52,35.77,75.13,44,0,5.17,SD,1970-01-01 00:00:01.604024949,Nearest Hotel Not Found


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
                            info_box_content= hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))